In [ ]:
# Testing LP Libraries: Pyomo, ROmodel, PuLP, OR Tools, Gekko, CVXPY, SciPy, ...

In [ ]:
# Diet Mix - Minimize the price but keep the nutrient values and balanced diet

In [1]:
# 1. Pyomo (Python Optimization Modeling Objects) Solver
!pip install pyomo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 694.5 kB/s eta 0:00:00


In [2]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123594 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [7]:
pip install scikit-glpk # GNU Linear Programming Kit

In [8]:
from pyomo.environ import *
import pyomo.environ as pyo

In [9]:
from glpk import glpk, GLPK

In [10]:
infinity = float('inf')
model = AbstractModel(name="Diet")

In [11]:
# Foods
model.F = Set()
# Nutrients
model.N = Set()

In [12]:
# Cost of each food
model.c    = Param(model.F, within=PositiveReals)
# Amount of nutrient in each food
model.a    = Param(model.F, model.N, within=NonNegativeReals)
# Lower and upper bound on each nutrient
model.Nmin = Param(model.N, within=NonNegativeReals, default=0.0)
model.Nmax = Param(model.N, within=NonNegativeReals, default=infinity)
# Volume per serving of food
model.V    = Param(model.F, within=PositiveReals)
# Maximum volume of food consumed
model.Vmax = Param(within=PositiveReals)

In [13]:
# Number of servings consumed of each food
model.x = Var(model.F, within=NonNegativeIntegers)

In [14]:
# Minimize the cost of food that is consumed
def cost_rule(model):
    return sum(model.c[i]*model.x[i] for i in model.F)
model.cost = Objective(rule=cost_rule)

In [15]:
# Limit nutrient consumption for each nutrient
def nutrient_rule(model, j):
    value = sum(model.a[i,j]*model.x[i] for i in model.F)
    return inequality(model.Nmin[j], value, model.Nmax[j])
model.nutrient_limit = Constraint(model.N, rule=nutrient_rule)

# Limit the volume of food consumed
def volume_rule(model):
    return sum(model.V[i]*model.x[i] for i in model.F) <= model.Vmax
model.volume = Constraint(rule=volume_rule)

In [16]:
import pandas as pd
from sklearn import datasets

In [19]:
instance = model.create_instance("diet.dat")
solver = SolverFactory('glpk')
result = solver.solve(instance, tee = False)
instance.display()
# The cheapest solution shows that for 2.15 EUR budget a person can get 1 Carb Mix, 1 Veggie Mix and 1 Quark)

Model Diet

  Variables:
    x : Size=8, Index=F
        Key                 : Lower : Value : Upper : Fixed : Stale : Domain
                   Carb mix :     0 :   1.0 :  None : False : False : NonNegativeIntegers
                    Jogurth :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                      Kakao :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                Lemon Juice :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        Milk Protein Pulver :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                      Quark :     0 :   1.0 :  None : False : False : NonNegativeIntegers
                  Sugar mix :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                 Veggie mix :     0 :   1.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  2.15

  Constraints:
    nutrient_limit :

In [ ]:
# 2. PuLP

In [ ]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 50.0 MB/s eta 0:00:00


In [ ]:
import pulp as pu
import pandas as pd
import os.path
import json
from pulp import *
import numpy as np, pandas as pd
import warnings

In [ ]:
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [ ]:
prob = LpProblem('Diet_Problem', LpMinimize)

In [ ]:
# Create some dictionaries in order to extract information from the diet table.

#Reading the data
df = pd.read_excel('diet.xlsx',nrows=9)
df.head(9)

,Foods,Price/Serving,Serving Size,Calories,Cholesterol (mg),Total_Fat (g),Sodium (mg),Carbohydrates (g),Dietary_Fiber (g),Protein (g),Vit_A (IU),Vit_C (IU),Calcium (mg),Iron (mg)
0,Cottage,1.39,250 g,170,0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,Tomaten,2.54,500 g,100,0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
2,Iceberg,0.99,1 Head,100,0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
3,Apple,0.72,1 Small,77,0,0.5,0.0,21.0,3.7,0.3,73.1,7.9,9.7,0.2
4,Fisch,0.99,180 g,170,0,0.5,1.1,26.7,2.7,1.2,92.3,10.4,6.8,0.4
5,Semmel,0.23,1,133,0,1.0,134.5,12.4,1.3,2.2,0.0,0.0,10.8,0.7
6,Milk,0.20,200 ml,80,0,1.0,132.5,11.8,1.1,2.3,0.0,0.0,26.2,0.8
7,Jogurth 0.1,0.50,180 g,70,0,3.3,68.9,12.4,0.6,1.1,2.9,0.1,6.7,0.5
8,Jogurth 3.6,0.60,180 g,110,0,3.1,75.4,9.6,0.5,0.5,35.2,0.9,3.1,0.1


In [ ]:
#List of food items
food = list(df.Foods)

#The list of items
count=pd.Series(range(1,len(food)+1))
print('List of different food items is here follows: -')
food_s = pd.Series(food)

#Convert to data frame
f_frame = pd.concat([count,food_s],axis=1,keys=['S.No','Food Items'])
f_frame

List of different food items is here follows: -


,S.No,Food Items
0,1,Cottage
1,2,Tomaten
2,3,Iceberg
3,4,Apple
4,5,Fisch
5,6,Semmel
6,7,Milk
7,8,Jogurth 0.1
8,9,Jogurth 3.6


In [ ]:
# Create a dictinary of costs for all food items
costs = dict(zip(food,df['Price/Serving']))

#Create a dictionary of calories for all items of food
calories = dict(zip(food,df['Calories']))

#Create a dictionary of cholesterol for all items of food
chol = dict(zip(food,df['Cholesterol (mg)']))

#Create a dictionary of total fat for all items of food
fat = dict(zip(food,df['Total_Fat (g)']))

#Create a dictionary of sodium for all items of food
sodium = dict(zip(food,df['Sodium (mg)']))

#Create a dictionary of carbohydrates for all items of food
carbs = dict(zip(food,df['Carbohydrates (g)']))

#Create a dictionary of dietary fiber for all items of food
fiber = dict(zip(food,df['Dietary_Fiber (g)']))

#Create a dictionary of protein for all food items
protein = dict(zip(food,df['Protein (g)']))

#Create a dictionary of vitamin A for all food items
vit_A = dict(zip(food,df['Vit_A (IU)']))

#Create a dictionary of vitamin C for all food items
vit_C = dict(zip(food,df['Vit_C (IU)']))

#Create a dictionary of calcium for all food items
calcium = dict(zip(food,df['Calcium (mg)']))

#Create a dictionary of iron for all food items
iron = dict(zip(food,df['Iron (mg)']))

In [ ]:
#We just run one of the dictionaries to see how these look like
iron

{'Cottage': 2.3,
 'Tomaten': 0.3,
 'Iceberg': 0.1,
 'Apple': 0.2,
 'Fisch': 0.4,
 'Semmel': 0.7,
 'Milk': 0.8,
 'Jogurth 0.1': 0.5,
 'Jogurth 3.6': 0.1}

In [ ]:
# A dictionary called 'food_vars' is created to contain the referenced Variables
food_vars = LpVariable.dicts("Food",food,lowBound=0,cat='Continuous')

In [ ]:
food_vars

{'Cottage': Food_Cottage,
 'Tomaten': Food_Tomaten,
 'Iceberg': Food_Iceberg,
 'Apple': Food_Apple,
 'Fisch': Food_Fisch,
 'Semmel': Food_Semmel,
 'Milk': Food_Milk,
 'Jogurth 0.1': Food_Jogurth_0.1,
 'Jogurth 3.6': Food_Jogurth_3.6}

In [ ]:
prob += lpSum([costs[i]*food_vars[i] for i in food])
prob

Diet_Problem:
MINIMIZE
0.72*Food_Apple + 1.39*Food_Cottage + 0.99*Food_Fisch + 0.99*Food_Iceberg + 0.5*Food_Jogurth_0.1 + 0.6*Food_Jogurth_3.6 + 0.2*Food_Milk + 0.23*Food_Semmel + 2.54*Food_Tomaten + 0.0
VARIABLES
Food_Apple Continuous
Food_Cottage Continuous
Food_Fisch Continuous
Food_Iceberg Continuous
Food_Jogurth_0.1 Continuous
Food_Jogurth_3.6 Continuous
Food_Milk Continuous
Food_Semmel Continuous
Food_Tomaten Continuous

In [ ]:
lpSum([food_vars[i]*calories[i] for i in food])

77*Food_Apple + 170*Food_Cottage + 170*Food_Fisch + 100*Food_Iceberg + 70*Food_Jogurth_0.1 + 110*Food_Jogurth_3.6 + 80*Food_Milk + 133*Food_Semmel + 100*Food_Tomaten + 0

In [ ]:
df[['Foods','Calories']]

,Foods,Calories
0,Cottage,170
1,Tomaten,100
2,Iceberg,100
3,Apple,77
4,Fisch,170
5,Semmel,133
6,Milk,80
7,Jogurth 0.1,70
8,Jogurth 3.6,110


In [ ]:
prob += lpSum([food_vars[x]*calories[x] for x in food]) >= 300, "CaloriesMinimum"
prob += lpSum([food_vars[x]*calories[x] for x in food]) <= 800, "CaloriesMaximum"
# prob

In [ ]:
prob

Diet_Problem:
MINIMIZE
0.72*Food_Apple + 1.39*Food_Cottage + 0.99*Food_Fisch + 0.99*Food_Iceberg + 0.5*Food_Jogurth_0.1 + 0.6*Food_Jogurth_3.6 + 0.2*Food_Milk + 0.23*Food_Semmel + 2.54*Food_Tomaten + 0.0
SUBJECT TO
CaloriesMinimum: 77 Food_Apple + 170 Food_Cottage + 170 Food_Fisch
 + 100 Food_Iceberg + 70 Food_Jogurth_0.1 + 110 Food_Jogurth_3.6
 + 80 Food_Milk + 133 Food_Semmel + 100 Food_Tomaten >= 300

CaloriesMaximum: 77 Food_Apple + 170 Food_Cottage + 170 Food_Fisch
 + 100 Food_Iceberg + 70 Food_Jogurth_0.1 + 110 Food_Jogurth_3.6
 + 80 Food_Milk + 133 Food_Semmel + 100 Food_Tomaten <= 800

VARIABLES
Food_Apple Continuous
Food_Cottage Continuous
Food_Fisch Continuous
Food_Iceberg Continuous
Food_Jogurth_0.1 Continuous
Food_Jogurth_3.6 Continuous
Food_Milk Continuous
Food_Semmel Continuous
Food_Tomaten Continuous

In [ ]:
#Carbohydrates' constraint
prob += lpSum([food_vars[x]*carbs[x] for x in food]) >= 40, "CarbsMinimum"
prob += lpSum([food_vars[x]*carbs[x] for x in food]) <= 200, "CarbsMaximum"

#Fat's constraint
prob += lpSum([food_vars[x]*fat[x] for x in food]) >= 3, "FatsMinimum"
prob += lpSum([food_vars[x]*fat[x] for x in food]) <= 50, "FatsMaximum"

#Protein's constraint
prob += lpSum([food_vars[x]*protein[x] for x in food]) >= 10, "ProteinsMinimum"
prob += lpSum([food_vars[x]*protein[x] for x in food]) <= 150, "ProteinsMaximum"

#Vit_A constraint
prob += lpSum([food_vars[x]*vit_A[x] for x in food]) >= 10, "Vit_A_Minimum"
prob += lpSum([food_vars[x]*vit_A[x] for x in food]) <= 10000, "Vit_A_Maximum"

In [ ]:
prob.solve()

1

In [ ]:
prob.solver

In [ ]:
LpStatus[prob.status]

'Optimal'

In [ ]:
# prob.variables()[1].varValue()

for var in prob.variables():
    print(f'Variable name: {var.name} , Variable value : {var.value()}\n')

print('\n')
print('*'*100)
print('\n')

#We can also see the slack variables of the constraints
for name, con in prob.constraints.items():
    print(f'constraint name:{name}, constraint value:{con.value()}\n')

print('*'*100)
print('\n')

## OBJECTIVE VALUE
print(f'OBJECTIVE VALUE IS: {round(prob.objective.value(),2)}')

Variable name: Food_Apple , Variable value : 0.0

Variable name: Food_Cottage , Variable value : 0.0017043324

Variable name: Food_Fisch , Variable value : 0.0

Variable name: Food_Iceberg , Variable value : 0.0

Variable name: Food_Jogurth_0.1 , Variable value : 0.0

Variable name: Food_Jogurth_3.6 , Variable value : 0.0

Variable name: Food_Milk , Variable value : 4.341898

Variable name: Food_Semmel , Variable value : 0.0

Variable name: Food_Tomaten , Variable value : 0.0



****************************************************************************************************


constraint name:CaloriesMinimum, constraint value:47.64157650799996

constraint name:CaloriesMaximum, constraint value:-452.358423492

constraint name:CarbsMinimum, constraint value:11.25757532064

constraint name:CarbsMaximum, constraint value:-148.74242467936

constraint name:FatsMinimum, constraint value:1.3432614659199995

constraint name:FatsMaximum, constraint value:-45.65673853408

constraint name:Prote

In [ ]:
# https://www.pyomo.org/
# https://medium.com/@chenycy/solve-optimization-problems-exploring-linear-programming-with-python-a299bcc9bdb8
# https://en.wikipedia.org/wiki/Pyomo
# https://www.kaggle.com/code/arifmeighan/basic-portfolio-optimization-with-pyopt
# https://github.com/Pyomo/PyomoGallery/wiki
# https://jckantor.github.io/CBE30338/06.04-Linear-Production-Model-in-Pyomo.html
# https://www.solvermax.com/blog/python-optimization-rosetta-stone
# https://www.solvermax.com/about
# https://github.com/SolverMax/Collated/blob/main/Diet/Diet-in-PuLP/Diet-in-PuLP.ipynb
# https://github.com/SolverMax/Collated/blob/main/Diet/Diet-in-OR-Tools/Diet-OR-Tools.ipynb
# https://www.pyomo.org/
# https://link.springer.com/article/10.1007/s11081-021-09703-2?fromPaywallRec=true
# https://en.wikipedia.org/wiki/Linear_programming#Solvers_and_scripting_(programming)_languages
